In [1]:
import env
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re
import lyricsgenius
import pandas as pd


ModuleNotFoundError: No module named 'env'

In [ ]:
genius = lyricsgenius.Genius(env.token)


In [ ]:
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
print(artist.songs)

Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]


In [ ]:
song = artist.songs[0]
print(song.lyrics)

Alexander All Alone Lyrics[Verse 1: Andy Shauf]
Alexander all alone
Smoking a cigarette
The last pack he’d ever buy
At least that’s what he said
He stood up to stretch his back
And fell down to the ground

[Verse 2: Andy Shauf]
Alexander all alone
'Till the neighbour caught a glimpse
Cried out for his wife
To call the ambulance
Alexander all alone
Felt them check his pulse
He heard them pronounce him dead
[Chorus]
Hell is found inside of me
And nothing else will set me free
If hell is found inside of me
Then open me up and spill me out

[Verse 3: Andy Shauf]
Alexander wondered why
No life flashed before his eyes
Why his soul did not depart
Why he found no peace of mind
Would it take a little while?
Was it the same for everyone?
Alexander realized

[Chorus 2]
That hell is found inside of me
And nothing else will set me free
If hell is found inside of me
Then open me up and spill me outEmbed


In [ ]:
df = pd.read_csv("charts.csv")
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [ ]:
df.shape

(330087, 7)

In [ ]:
#convert date to datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [ ]:
first_date = pd.DataFrame(df.groupby(['artist', 'song']).date.min())

In [ ]:
# add a column for year and decade
first_date['year'] = first_date.date.dt.year
first_date['decade'] = first_date.date.dt.year - (first_date.date.dt.year % 10)
first_date.reset_index(inplace=True)
first_date.head()

,artist,song,date,year,decade
0,"""Groove"" Holmes",Misty,1966-06-25,1966,1960
1,"""Groove"" Holmes",What Now My Love,1966-10-01,1966,1960
2,"""Little"" Jimmy Dickens",May The Bird Of Paradise Fly Up Your Nose,1965-10-16,1965,1960
3,"""Pookie"" Hudson",I Know I Know,1963-05-25,1963,1960
4,"""Weird Al"" Yankovic",Amish Paradise,1996-03-30,1996,1990


In [ ]:
first_date.shape

(29681, 5)

In [ ]:
def prep_data(df):
    '''
    get unique artist song combos
    make column for year and decade
    change song to title
    '''
    # convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    # rename song column to title
    df.rename(columns={'song': 'title'}, inplace=True)
    # subset data to make only unique artist title combos while keeping earliest date
    df = pd.DataFrame(df.groupby(['artist', 'title']).date.min())
    # make new column for year and decade
    df['year'] = df.date.dt.year
    df['decade'] = df.date.dt.year - (df.date.dt.year % 10)
    # reset index
    df.reset_index(inplace=True)
    return df

In [ ]:
df = prep_data(pd.read_csv("charts.csv"))
df.head()

,artist,title,date,year,decade
0,"""Groove"" Holmes",Misty,1966-06-25,1966,1960
1,"""Groove"" Holmes",What Now My Love,1966-10-01,1966,1960
2,"""Little"" Jimmy Dickens",May The Bird Of Paradise Fly Up Your Nose,1965-10-16,1965,1960
3,"""Pookie"" Hudson",I Know I Know,1963-05-25,1963,1960
4,"""Weird Al"" Yankovic",Amish Paradise,1996-03-30,1996,1990


In [ ]:
df.shape

(29681, 5)

In [ ]:

api = lyricsgenius.Genius(env.token, verbose=False)

def prep_songs():
    songs = pd.read_csv('charts.csv')
    songs['date'] = pd.to_datetime(songs.date)
    songs = songs.rename(columns={'song': 'title'})
    songs = pd.DataFrame(songs.groupby(['title', 'artist']).min().date).reset_index()
    return songs

def get_lyrics(title, artist):
    song = api.search_song(title, artist)
    lyrics = song.lyrics.replace('\n', ' ')
    # TODO handle verse and chorus tags
    return lyrics

songs = prep_songs()

for i in range(len(songs)-1, 0 , -1):
    
    title = songs.iloc[[i]].title.values[0]
    artist = songs.iloc[[i]].artist.values[0]
    
    try:
        songs.loc[i, 'lyrics'] = get_lyrics(title, artist)
        songs.loc[i, 'status'] = 'lyrics acquired'
    except: 
        songs.loc[i, 'status'] = 'an error ocurred'
    # save every 100 songs
    if i % 100 == 0:
        songs.to_csv('songs.csv')
    
    print(f'{round((i / len(songs) * 100), 2)}%\t complete', end='\r')

In [ ]:
songs.to_csv('songs.csv')

In [ ]:
songs.head()

,title,artist,date,status,lyrics
0,"""B"" Girls",Young And Restless,1990-05-05,NaN,NaN
1,"""Cherry Cherry"" from Hot August Night",Neil Diamond,1973-03-17,an error ocurred,NaN
2,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),1973-10-13,an error ocurred,NaN
3,"""Joy"" Pt. I",Isaac Hayes,1973-12-22,lyrics acquired,"12/12 Songs LyricsAAlex Turner - ""Stuck on the..."
4,"""Roots"" Medley",Quincy Jones,1977-03-05,lyrics acquired,Ulysses (Chap. 15 - Circe) LyricsThe Mabbot st...


In [ ]:
songs.shape

(29681, 5)

In [ ]:
songs.isnull().sum()

title        0
artist       0
date         0
status       1
lyrics    1917
dtype: int64

In [ ]:
songs[songs.status== 'an error ocurred']

,title,artist,date,status,lyrics
1,"""Cherry Cherry"" from Hot August Night",Neil Diamond,1973-03-17,an error ocurred,NaN
2,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),1973-10-13,an error ocurred,NaN
5,"""Yep!""",Duane Eddy His Twangy Guitar And The Rebels,1959-03-30,an error ocurred,NaN
15,'98 Thug Paradise,"Tragedy, Capone, Infinite",1998-09-19,an error ocurred,NaN
26,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968-01-13,an error ocurred,NaN
...,...,...,...,...,...
29671,pride.is.the.devil,J. Cole & Lil Baby,2021-05-29,an error ocurred,NaN
29672,punchin'.the.clock,J. Cole,2021-05-29,an error ocurred,NaN
29675,the.climb.back,J. Cole,2021-05-29,an error ocurred,NaN
29676,transparentsoul,Willow Featuring Travis Barker,2021-06-26,an error ocurred,NaN


# Prep data

In [ ]:
songs[songs.status== 'lyrics acquired'].head(10)

,title,artist,date,status,lyrics
3,"""Joy"" Pt. I",Isaac Hayes,1973-12-22,lyrics acquired,"12/12 Songs LyricsAAlex Turner - ""Stuck on the..."
4,"""Roots"" Medley",Quincy Jones,1977-03-05,lyrics acquired,Ulysses (Chap. 15 - Circe) LyricsThe Mabbot st...
6,#1,Nelly,2001-10-20,lyrics acquired,#1 LyricsUh uh uh I just gotta bring it to the...
7,#1 Dee Jay,Goody Goody,1978-11-18,lyrics acquired,Lbo3d l’akhar LyricsVerse 1: (Omar Cravate) B...
8,#9 Dream,John Lennon,1974-12-21,lyrics acquired,#9 Dream Lyrics[Verse 1] So long ago Was it in...
9,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,lyrics acquired,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,..."
10,#SELFIE,The Chainsmokers,2014-03-15,lyrics acquired,#SELFIE Lyrics[Verse 1] When Jason was at the ...
11,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,lyrics acquired,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...
12,$ave Dat Money,Lil Dicky Featuring Fetty Wap & Rich Homie Quan,2015-10-10,lyrics acquired,$ave Dat Money Lyrics[Intro: Lil Dicky & Fetty...
13,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,2002-10-26,lyrics acquired,"’03 Bonnie & Clyde Lyrics[Intro: JAY-Z] Uhh, u..."


In [ ]:
songs.title[8] in songs.lyrics[8]

True

In [ ]:
# is the title in the lyrics?
songs['correct'] = (songs.title in songs.lyrics)
songs.head()



TypeError: unhashable type: 'Series'

In [ ]:
# stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords[:5]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/teceno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we']

In [ ]:
df = pd.read_csv('songs_0526.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'songs_0526.csv'